## <center><h1 style="font-size:32px;">Import Libraries</h1></center>

In [1]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from urllib.request import Request, urlopen
import re
import pandas as pd
import time

## <center><h1 style="font-size:32px;">Create Soup Object</h1></center>

In [3]:
req = Request('https://gatherer.wizards.com/Pages/Search/Default.aspx?page=0&action=advanced&cmc=+%3E=[0]')
html_page = urlopen(req).read()
soup = BeautifulSoup(html_page, 'html.parser')

In [6]:
soup#.prettify()

<?xml version="1.0" encoding="utf-8" ?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>
	Card Search - Search: +&gt;=0 - Gatherer - Magic: The Gathering
</title><link href="/Images/favicon.ico" rel="shortcut icon"/><meta content="Gatherer is the Magic Card Database. Search for the perfect addition to your deck. Browse through cards from Magic's entire history. See cards from the most recent sets and discover what players just like you are saying about them." name="description"/><meta content="monitor, gatherer, magic cards, magic the gathering, black lotus, magic: the gathering, wizards of the coast, wizards, trading card game, trading cards, collectible card game, tcg, ccg, magic sets, game, multiplayer, hobby" name="keywords"/><meta content="#191919" name="theme-color"/><meta content="#191919" name="msapplicatio

## <center><h1 style="font-size:32px;">Exploring the Soup for Data</h1></center>

In [13]:
soup = soup.table

In [14]:
#locate 1st card item
tag = soup.table
#Locate card name
tag.td.img['alt']

'"Ach! Hans, Run!"'

In [34]:
#move to next card
tag = tag.next_sibling.next_sibling
tag

<table>
<tr class="cardItem oddItem">
<td class="leftCol">
<div class="clear"></div>
<a href="../Card/Details.aspx?multiverseid=580654" id="ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ctl00_listRepeater_ctl01_cardImageLink" onclick="return CardLinkAction(event, this, 'SameWindow');">
<img alt='"Lifetime" Pass Holder' border="0" height="132" id="ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ctl00_listRepeater_ctl01_cardImage" src="../../Handlers/Image.ashx?multiverseid=580654&amp;type=card" style="border-radius:6px;-webkit-border-radius:6px;-moz-border-radius:6px;" width="95"/>
</a>
<div class="clear"></div>
</td>
<td class="middleCol">
<div class="clear"></div>
<div class="cardInfo">
<span class="cardTitle">
<a href="../Card/Details.aspx?multiverseid=580654" id="ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ctl00_listRepeater_ctl01_cardTitle" onclick="return CardLinkAction(event, this, 'SameWindow');">"Lifetime" Pass Holder</a></span> <span class="manaCost">
<img al

In [35]:
#check card name
tag.td.img['alt']

'"Lifetime" Pass Holder'

In [28]:
#Locate card type
tag.find('td',{'class':'middleCol'}).find('span',{'class':'typeLine'}).text.strip()

'Enchantment'

In [10]:
#locate converted mana cost
tag.find('td',{'class':'middleCol'}).find('span',{'class':'convertedManaCost'}).text

'6'

In [13]:
#extract mana cost
tag2 = tag.find('td',{'class':'middleCol'}).find('span',{'class':'manaCost'})
for i in tag2.find_all('img'):
    print(i['alt'])

2
Red
Red
Green
Green


In [14]:
#get rules text(i replaced this with a better method later to account for icons in the rules text)
tag.find('div',{'class':'rulesText'}).p.text

'At the beginning of your upkeep, you may say "Ach Hans, run It\'s the . . ." and the name of a creature card. If you do, search your library for a card with that name, put it onto the battlefield, then shuffle. That creature gains haste. Exile it at the beginning of the next end step.'

In [20]:
#get link for further card details
tag.find('td',{'class':'leftCol'}).a['href'][2:]

'/Card/Details.aspx?multiverseid=73935'

In [83]:
req = Request('https://gatherer.wizards.com/Pages' + tag.find('td',{'class':'leftCol'}).a['href'][2:])

html_page = urlopen(req).read()
soup2 = BeautifulSoup(html_page, 'html.parser')

tag2 = soup2
tag2

<?xml version="1.0" encoding="utf-8" ?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>
	Abandon Reason (Eldritch Moon) - Gatherer - Magic: The Gathering
</title><link href="/Images/favicon.ico" rel="shortcut icon"/><meta content="Gatherer is the Magic Card Database. Search for the perfect addition to your deck. Browse through cards from Magic's entire history. See cards from the most recent sets and discover what players just like you are saying about them." name="description"/><meta content="monitor, gatherer, magic cards, magic the gathering, black lotus, magic: the gathering, wizards of the coast, wizards, trading card game, trading cards, collectible card game, tcg, ccg, magic sets, game, multiplayer, hobby" name="keywords"/><meta content="#191919" name="theme-color"/><meta content="#191919" name="msapplicati

In [73]:
#extract card text while accounting for icons
text = tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_textRow'}).find_all('div',{'class':'cardtextbox'})
rules_text = ''
for i in text:
    for y in i:
        try:
            rules_text += y['alt']
        except:
            rules_text += y.text
            
rules_text

'At the beginning of your upkeep, you may say "Ach Hans, run It\'s the . . ." and the name of a creature card. If you do, search your library for a card with that name, put it onto the battlefield, then shuffle. That creature gains haste. Exile it at the beginning of the next end step.'

In [74]:
# get card set
tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_currentSetSymbol'}).find_all('a')[1].text

'Unhinged'

In [75]:
#get card rarity
tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_rarityRow'}).find('div',{'class':'value'}).span.text

'Rare'

In [70]:
# get all sets cards appears in(results in error if only 1 set)
[i.a.img['title'] for i in tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_otherSetsValue'}).find_all('div')]

AttributeError: 'NoneType' object has no attribute 'find_all'

In [78]:
tag2.find('li',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContentAnchors_DetailsAnchors_Printings'}).a['href']

'Printings.aspx?multiverseid=73935'

In [84]:
# make another soup of specific cards legality
req = Request('https://gatherer.wizards.com/Pages/Card/'
              + tag2.find('li',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContentAnchors_DetailsAnchors_Printings'}).a['href'])
html_page = urlopen(req).read()
soup3 = BeautifulSoup(html_page, 'html.parser')
tag3 = soup3

In [88]:
#create list of legal formats
[i.td.text.strip() for i in tag3.find_all('tr',{'class':'cardItem isAlternating %>'})]

['Pioneer', 'Modern', 'Legacy', 'Vintage', 'Commander']

In [7]:
#create dictionary to be converted to dataframe later
x = {'name':[],'card_type':[],'mana_cost':[],'converted_cost':[], 'rules_text':[],'rarity':[],'latest_set':[],'all_sets':[],'legal_formats':[]}

In [15]:
#extract all data on gatherer's first page. See bottom for fully commented version
tag = soup.table
for i in range(100):
    x['name'].append(tag.td.img['alt'])
    x['card_type'].append(tag.find('td',{'class':'middleCol'}).find('span',{'class':'typeLine'}).text)
    tag2 = tag.find('td',{'class':'middleCol'}).find('span',{'class':'manaCost'})
    x['mana_cost'].append([i['alt'] for i in tag2.find_all('img')])
    x['converted_cost'].append(tag.find('td',{'class':'middleCol'}).find('span',{'class':'convertedManaCost'}).text)
    req = Request('https://gatherer.wizards.com/Pages' + tag.find('td',{'class':'leftCol'}).a['href'][2:])
    html_page = urlopen(req).read()
    soup2 = BeautifulSoup(html_page, 'html.parser')
    #soup2 = soup2.table
    tag2 = soup2#.table
    try:
        text = tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_textRow'}).find_all('div',{'class':'cardtextbox'})
        rules_text = ''
        for i in text:
            for y in i:
                try:
                    rules_text += y['alt']
                    rules_text += ' '
                except:
                    rules_text += y.text
        x['rules_text'].append(rules_text)
    except:
        x['rules_text'].append('')
    try:
        x['rarity'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_rarityRow'}).find('div',{'class':'value'}).span.text)
    except:
        x['rarity'].append('')
    #x['rarity'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_rarityRow'}).find('div',{'class':'value'}).span.text)
    try:
        x['latest_set'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_currentSetSymbol'}).find_all('a')[1].text)
    except:
        x['latest_set'].append('')
    try:
        x['all_sets'].append([i.a.img['title'] for i in tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_otherSetsValue'}).find_all('div')])
    except:
        x['all_sets'].append(1)
    req = Request('https://gatherer.wizards.com/Pages/Card/'
              + tag2.find('li',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContentAnchors_DetailsAnchors_Printings'}).a['href'])
    html_page = urlopen(req).read()
    soup3 = BeautifulSoup(html_page, 'html.parser')
    tag3 = soup3
    x['legal_formats'].append([i.td.text.strip() for i in tag3.find_all('tr',{'class':'cardItem isAlternating %>'})])
    tag = tag.next_sibling.next_sibling

In [16]:
#make dataframe
df1 = pd.DataFrame(x)
#get power and toughness(produces weird results on non creature cards. needs further cleaning)
df1['power'] = df1['card_type'].str[-4]
df1['toughness'] = df1['card_type'].str[-2]
#df1['card_type'][1]
df1

,name,card_type,mana_cost,converted_cost,rules_text,rarity,latest_set,all_sets,legal_formats,power,toughness
0,"""Ach! Hans, Run!""",\r\n Enchantmen...,"[2, Red, Red, Green, Green]",6,"At the beginning of your upkeep, you may say ""...",Rare,Unhinged,1,[],,
1,"""Lifetime"" Pass Holder",\r\n Creature ...,[Black],1,"""Lifetime"" Pass Holder enters the battlefield ...",Rare,Unfinity,1,"[Legacy, Vintage, Commander]",2,1
2,"""Rumors of My Death . . .""",\r\n Enchantmen...,"[2, Black]",3,"3 Black , Exile a permanent you control with a...",Uncommon,Unstable,1,[],,
3,_____,\r\n Creature ...,"[1, Blue]",2,1 : This card's name becomes the card name of ...,Uncommon,Unhinged,1,[],1,1
4,_____ _____ _____ Trespasser,\r\n Creature ...,"[1, Blue]",2,"When this creature enters the battlefield, you...",Uncommon,Unfinity,1,"[Legacy, Vintage, Commander]",2,1
...,...,...,...,...,...,...,...,...,...,...,...
95,Academy Ruins,\r\n Legendary ...,[],0,"Tap : Add Colorless .1 Blue , Tap : Put target...",Rare,Double Masters,"[Time Spiral (Rare), Modern Masters (Rare), Do...","[Modern, Legacy, Vintage, Commander]",,
96,Academy Wall,\r\n Creature ...,"[2, Blue]",3,DefenderWhenever you cast an instant or sorcer...,Common,Dominaria United,1,"[Standard, Brawl, Pioneer, Modern, Legacy, Vin...",0,5
97,Accelerate,\r\n Instant\r\...,"[1, Red]",2,Target creature gains haste until end of turn....,Common,Torment,1,"[Legacy, Vintage, Commander, Pauper]",,
98,Accelerated Mutation,\r\n Instant\r\...,"[3, Green, Green]",5,"Target creature gets +X/+X until end of turn, ...",Common,Scourge,1,"[Legacy, Vintage, Commander, Pauper]",,


In [8]:
df1 = pd.read_csv('MTGscraped.csv')

req = Request('https://gatherer.wizards.com/Pages/Search/Default.aspx?page=99&action=advanced&cmc=+%3E=[0]')
html_page = urlopen(req).read()
soup = BeautifulSoup(html_page, 'html.parser')

for i in range(100):
    x['name'].append(tag.td.img['alt'])
    x['card_type'].append(tag.find('td',{'class':'middleCol'}).find('span',{'class':'typeLine'}).text)
    tag2 = tag.find('td',{'class':'middleCol'}).find('span',{'class':'manaCost'})
    x['mana_cost'].append([i['alt'] for i in tag2.find_all('img')])
    x['converted_cost'].append(tag.find('td',{'class':'middleCol'}).find('span',{'class':'convertedManaCost'}).text)
    req = Request('https://gatherer.wizards.com/Pages' + tag.find('td',{'class':'leftCol'}).a['href'][2:])
    html_page = urlopen(req).read()
    soup2 = BeautifulSoup(html_page, 'html.parser')
    #soup2 = soup2.table
    tag2 = soup2#.table
    try:
        text = tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_textRow'}).find_all('div',{'class':'cardtextbox'})
        rules_text = ''
        for i in text:
            for y in i:
                try:
                    rules_text += y['alt']
                    rules_text += ' '
                except:
                    rules_text += y.text
        x['rules_text'].append(rules_text)
    except:
        x['rules_text'].append('')
    try:
        x['rarity'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_rarityRow'}).find('div',{'class':'value'}).span.text)
    except:
        x['rarity'].append('')
    #x['rarity'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_rarityRow'}).find('div',{'class':'value'}).span.text)
    try:
        x['latest_set'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_currentSetSymbol'}).find_all('a')[1].text)
    except:
        x['latest_set'].append('')
    try:
        x['all_sets'].append([i.a.img['title'] for i in tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_otherSetsValue'}).find_all('div')])
    except:
        x['all_sets'].append(1)
    req = Request('https://gatherer.wizards.com/Pages/Card/'
              + tag2.find('li',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContentAnchors_DetailsAnchors_Printings'}).a['href'])
    html_page = urlopen(req).read()
    soup3 = BeautifulSoup(html_page, 'html.parser')
    tag3 = soup3
    x['legal_formats'].append([i.td.text.strip() for i in tag3.find_all('tr',{'class':'cardItem isAlternating %>'})])
    tag = tag.next_sibling.next_sibling
df2 = pd.DataFrame(x)
#get power and toughness(produces weird results on non creature cards. needs further cleaning)
df2['power'] = df2['card_type'].str[-4]
df2['toughness'] = df2['card_type'].str[-2]
    
df1 = pd.concat([df1,df2], axis = 0)
df1 = df1.reset_index(drop = True)
df1.to_csv('MTGscraped.csv', index=False)

HTTPError: HTTP Error 500: Internal Server Error

In [20]:
df1.dtypes

name              object
card_type         object
mana_cost         object
converted_cost    object
rules_text        object
rarity            object
latest_set        object
all_sets          object
legal_formats     object
power             object
toughness         object
dtype: object

In [9]:
# This block is where i have played with the code trying to bypass Gatherer's antiscraping features

'''

retry_strategy = Retry(
    total=5,
    backoff_factor=2,
    status_forcelist=[ 500, 502, 503, 504 ])
adapter = HTTPAdapter(max_retries=retry_strategy)

http = requests.Session()
http.mount("https://", adapter)
http.mount("http://", adapter)
'''

#repeat process for multiple pages of cards. takes approx 9 hours on my machine
df1 = pd.read_csv('MTGscraped.csv')
z = 100
headers = {'User-Agent': 'Mozilla/5.0'}
for i in range(159):
    x = {'name':[],'card_type':[],'mana_cost':[],'converted_cost':[], 'rules_text':[],'rarity':[],'latest_set':[],'all_sets':[],'legal_formats':[]}
    z += 1
    req = Request('https://gatherer.wizards.com/Pages/Search/Default.aspx?page='+str(z)+'&action=advanced&cmc=+%3E=[0]', headers = headers)
    html_page = urlopen(req).read()
    soup = BeautifulSoup(html_page, 'html.parser')
    soup = soup.table
    tag = soup.table
    for i in range(100):
        x['name'].append(tag.td.img['alt'])
        x['card_type'].append(tag.find('td',{'class':'middleCol'}).find('span',{'class':'typeLine'}).text.strip())
        tag2 = tag.find('td',{'class':'middleCol'}).find('span',{'class':'manaCost'})
        x['mana_cost'].append([i['alt'] for i in tag2.find_all('img')])
        x['converted_cost'].append(tag.find('td',{'class':'middleCol'}).find('span',{'class':'convertedManaCost'}).text)
        #time.sleep(5)
        req = Request('https://gatherer.wizards.com/Pages' + tag.find('td',{'class':'leftCol'}).a['href'][2:])#, headers = headers)
        html_page = urlopen(req).read()
        soup2 = BeautifulSoup(html_page, 'html.parser')
        #soup2 = soup2.table
        tag2 = soup2#.table
        try:
            text = tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_textRow'}).find_all('div',{'class':'cardtextbox'})
            rules_text = ''
            for i in text:
                for y in i:
                    try:
                        rules_text += y['alt']
                        rules_text += ' '
                    except:
                        rules_text += y.text
            x['rules_text'].append(rules_text)
        except:
            x['rules_text'].append('')
        try:
            x['rarity'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_rarityRow'}).find('div',{'class':'value'}).span.text)
        except:
            x['rarity'].append('')
        #x['rarity'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_rarityRow'}).find('div',{'class':'value'}).span.text)
        try:
            x['latest_set'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_currentSetSymbol'}).find_all('a')[1].text)
        except:
            x['latest_set'].append('')
        try:
            x['all_sets'].append([i.a.img['title'] for i in tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_otherSetsValue'}).find_all('div')])
        except:
            x['all_sets'].append(1)
        #time.sleep(5)
        req = Request('https://gatherer.wizards.com/Pages/Card/'
                  + tag2.find('li',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContentAnchors_DetailsAnchors_Printings'}).a['href'])#, headers = headers)
        html_page = urlopen(req).read()
        soup3 = BeautifulSoup(html_page, 'html.parser')
        tag3 = soup3
        x['legal_formats'].append([i.td.text.strip() for i in tag3.find_all('tr',{'class':'cardItem isAlternating %>'})])
        tag = tag.next_sibling.next_sibling
    #make data frame
    df2 = pd.DataFrame(x)
    #get power and toughness(produces weird results on non creature cards. needs further cleaning)
    df2['power'] = df2['card_type'].str[-4]
    df2['toughness'] = df2['card_type'].str[-2]
    
    df1 = pd.concat([df1,df2], axis = 0)
    df1 = df1.reset_index(drop = True)
    df1.to_csv('MTGscraped.csv', index=False)
    

HTTPError: HTTP Error 500: Internal Server Error

In [ ]:
#make data frame
df2 = pd.DataFrame(x)
#get power and toughness(produces weird results on non creature cards. needs further cleaning)
df2['power'] = df2['card_type'].str[-4]
df2['toughness'] = df2['card_type'].str[-2]
df2

In [ ]:
#merge 1st page results with other pages
df3 = pd.concat([df1,df2], axis = 0)
df3 = df3.reset_index(drop = True)
df3

In [ ]:
#make a csv file
df3.to_csv('MTGscraped.csv', index=False)

## <center><h1 style="font-size:32px;">Clean and Commented Version</h1></center>

In [3]:
#This code got me about 1 third of the data from gatherer before the site blocked my scraping


# Initialize the value of `z` to 0 and create a dictionary to store the scraped data
z = 0
x = {'name':[],'card_type':[],'mana_cost':[],'converted_cost':[], 'rules_text':[],'rarity':[],'latest_set':[],'all_sets':[],'legal_formats':[],'power':[],'toughness':[]}
#'''IMPORTANT!!! IMPORTANT!!! IMPORTANT!!! '''
# comment out the below line of code for every run after the first or you will lose your data!
df1 = pd.DataFrame(x)
#'''IMPORTANT!!! IMPORTANT!!! IMPORTANT!!! '''
# comment out the below line of code for every run after the first or you will lose your data!
df1.to_csv('MTGscraped2.csv', index=False)
df1 = pd.read_csv('MTGscraped2.csv')
# Loop over `range(number of pages)` to scrape data from 257 pages
for i in range(260):
    x = {'name':[],'card_type':[],'mana_cost':[],'converted_cost':[], 'rules_text':[],'rarity':[],'latest_set':[],'all_sets':[],'legal_formats':[]}
    # Increment `z` by 1 on each iteration
    z += 1
    
    # Make a request to the URL with the current page number
    req = Request('https://gatherer.wizards.com/Pages/Search/Default.aspx?page='+str(z)+'&action=advanced&cmc=+%3E=[0]')
    
    # Read the HTML content of the page and parse it with BeautifulSoup
    html_page = urlopen(req).read()
    soup = BeautifulSoup(html_page, 'html.parser')
    soup = soup.table
    
    # Get the main table from the parsed HTML
    tag = soup.table
    
    # Loop over `range(100)` to scrape data for 100 cards per page
    for i in range(100):
        # Append the name of the card to the 'name' key of the dictionary
        x['name'].append(tag.td.img['alt'])
        
        # Append the type of the card to the 'card_type' key of the dictionary
        x['card_type'].append(tag.find('td',{'class':'middleCol'}).find('span',{'class':'typeLine'}).text.strip())
        
        # Get the mana cost of the card and append it to the 'mana_cost' key of the dictionary
        tag2 = tag.find('td',{'class':'middleCol'}).find('span',{'class':'manaCost'})
        x['mana_cost'].append([i['alt'] for i in tag2.find_all('img')])
        
        # Append the converted mana cost of the card to the 'converted_cost' key of the dictionary
        x['converted_cost'].append(tag.find('td',{'class':'middleCol'}).find('span',{'class':'convertedManaCost'}).text)
        
        # Make another request to get the details of the card
        req = Request('https://gatherer.wizards.com/Pages' + tag.find('td',{'class':'leftCol'}).a['href'][2:])
        html_page = urlopen(req).read()
        soup2 = BeautifulSoup(html_page, 'html.parser')
        tag2 = soup2
        
        # Try to build the rules text of the card and append it to the 'rules_text' key of the dictionary
        try:
            text = tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_textRow'}).find_all('div',{'class':'cardtextbox'})
            rules_text = ''
            for i in text:
                for y in i:
                    try:
                        rules_text += y['alt']
                        rules_text += ' '
                    except:
                        rules_text += y.text
            x['rules_text'].append(rules_text)
        except:
            x['rules_text'].append('')
        # Try to find the rarity of the card
        # Try to find the 'rarity' and append it to the list x['rarity']
        try:
            x['rarity'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_rarityRow'}).find('div',{'class':'value'}).span.text)
        # If the 'rarity' is not found, append an empty string instead
        except:
            x['rarity'].append('')

        # Try to find the 'latest_set' and append it to the list x['latest_set']
        try:
            x['latest_set'].append(tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_currentSetSymbol'}).find_all('a')[1].text)
        # If the 'latest_set' is not found, append an empty string instead
        except:
            x['latest_set'].append('')

        # Try to find the 'all_sets' and append it to the list x['all_sets']
        try:
            x['all_sets'].append([i.a.img['title'] for i in tag2.find('div',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_otherSetsValue'}).find_all('div')])
        # If the 'all_sets' is not found, append the value '1' instead
        except:
            x['all_sets'].append(1)

        # Send a request to the URL specified and read the response
        req = Request('https://gatherer.wizards.com/Pages/Card/' + tag2.find('li',{'id':'ctl00_ctl00_ctl00_MainContent_SubContent_SubContentAnchors_DetailsAnchors_Printings'}).a['href'])
        html_page = urlopen(req).read()

        # Create a BeautifulSoup object from the response
        soup3 = BeautifulSoup(html_page, 'html.parser')
        tag3 = soup3

        # Find all elements with the class 'cardItem isAlternating %>' and append their td text as a list to x['legal_formats']
        x['legal_formats'].append([i.td.text.strip() for i in tag3.find_all('tr',{'class':'cardItem isAlternating %>'})])
        #this value is fixed later in the code
        x['power'].append(0)
        #this value is fixed later in the code
        x['toughness'].append(0)

        # Move to the next sibling of the current tag
        tag = tag.next_sibling.next_sibling
    
    df2 = pd.DataFrame(x)
    #get power and toughness(produces weird results on non creature cards. needs further cleaning)
    df2['power'] = df2['card_type'].str[-4]
    df2['toughness'] = df2['card_type'].str[-2]
    #combine saved data with new data
    df1 = pd.concat([df1,df2], axis = 0)
    df1 = df1.reset_index(drop = True)
    #save combined data
    df1.to_csv('MTGscraped2.csv', index=False)



HTTPError: HTTP Error 500: Internal Server Error